In [1]:
import numpy as np
import pandas as pd 
import astropy.io.ascii as ascii
from astropy.table import Table
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky
import PIL
from PIL import Image
import sys
import os
import matplotlib.image as mp

In [2]:
def match1(poi, catalogue):
    
    x=poi['X_WORLD']
    y=poi['Y_WORLD']

    coord1 = SkyCoord(ra=[x]*u.degree, dec=[y]*u.degree)
    coord2 = SkyCoord(ra=catalogue['X_WORLD']*u.degree, dec=catalogue['Y_WORLD']*u.degree)

    idx, d2d, d3d = coord1.match_to_catalog_sky(coord2)
    max_sep = 1.0 * u.arcsec
    sep_constraint = d2d < max_sep
    global catalogue_match
    catalogue_match = catalogue.iloc[idx[sep_constraint]]
    
    
def match2(catalogue1, catalogue2):
    xworld1 = catalogue1['X_WORLD']
    yworld1 = catalogue1['Y_WORLD']

    xworld2 = catalogue2['X_WORLD']
    yworld2 = catalogue2['Y_WORLD']

    coord1 = SkyCoord(ra=xworld1*u.degree, dec=yworld1*u.degree)
    coord2 = SkyCoord(ra=xworld2*u.degree, dec=yworld2*u.degree)

    idx, d2d, d3d = coord1.match_to_catalog_sky(coord2)

    max_sep = 1.0 * u.arcsec

    sep_constraint = d2d < max_sep

    global data1_12_matches
    try: data1_12_matches = catalogue1[sep_constraint]
    except IndexError: 
        pass 
    
    global data2_12_matches
    data2_12_matches = catalogue2.iloc[idx[sep_constraint]]

    
def match3(catalogue1, catalogue2, catalogue3):
    xworld1 = catalogue1['X_WORLD']
    yworld1 = catalogue1['Y_WORLD']

    xworld2 = catalogue2['X_WORLD']
    yworld2 = catalogue2['Y_WORLD']
    
    xworld3 = catalogue3['X_WORLD']
    yworld3 = catalogue3['Y_WORLD']
    
    coord1 = SkyCoord(ra=xworld1*u.degree, dec=yworld1*u.degree)
    coord2 = SkyCoord(ra=xworld2*u.degree, dec=yworld2*u.degree)

    
    idx, d2d, d3d = coord1.match_to_catalog_sky(coord2)
    
    max_sep = 1.0 * u.arcsec

    sep_constraint = d2d < max_sep
    
    data1_12_matches = catalogue1[sep_constraint]
    data2_12_matches = catalogue2.iloc[idx[sep_constraint]]
    
    xworld1_12 = data1_12_matches['X_WORLD']
    yworld1_12 = data1_12_matches['Y_WORLD']
    
    coord1_12 = SkyCoord(ra=xworld1_12*u.degree, dec=yworld1_12*u.degree)
    coord3 = SkyCoord(ra=xworld3*u.degree, dec=yworld3*u.degree)
    
    idx, d2d, d3d = coord1_12.match_to_catalog_sky(coord3)
    
    max_sep = 1.0 * u.arcsec

    sep_constraint = d2d < max_sep
    
    global data1_123_matches
    data1_123_matches = data1_12_matches[sep_constraint]
    global data2_123_matches
    data2_123_matches = data2_12_matches[sep_constraint]
    global data3_123_matches
    data3_123_matches = catalogue3.iloc[idx[sep_constraint]]

In [ ]:
r1 = ascii.read('GRB210605A5_20210608_g_stack_ext5.resamp_sub_withnans_SUB.cat')
r2 = ascii.read('GRB210605A5_20210609_g_stack_ext5.resamp_sub_withnans_SUB.cat')
r3 = ascii.read('GRB210605A5_20210610_g_stack_ext5.resamp_sub_withnans_SUB.cat')
r4 = ascii.read('GRB210605A5_20210608_i_stack_ext5.resamp_sub_withnans_SUB.cat')
r5 = ascii.read('GRB210605A5_20210608_i_stack_ext5.resamp_sub_withnans_SUB.cat')
r6 = ascii.read('GRB210605A5_20210608_i_stack_ext5.resamp_sub_withnans_SUB.cat')

#reads in the cat files as panda dataframes
df1 = pd.DataFrame(r1.as_array())
df2 = pd.DataFrame(r2.as_array())
df3 = pd.DataFrame(r3.as_array())
df4 = pd.DataFrame(r4.as_array())
df5 = pd.DataFrame(r5.as_array())
df6 = pd.DataFrame(r6.as_array())

print('data points in first file:', len(r1))
print('data points in second file:', len(r2))
print('data points in third file:', len(r3))
print('data points in fourth file:', len(r4))
print('data points in fifth file:', len(r5))
print('data points in sixth file:', len(r6))

#getting rid of the points in the sub images with a mag_auto of greater than 20 as these are errors
df1f = df1[df1['MAG_AUTO'] < 20]
df2f = df2[df2['MAG_AUTO'] < 20]
df3f = df3[df3['MAG_AUTO'] < 20]
 #same thing but for the science images 
df4f = df4[df4['MAG_AUTO'] < 20]
df5f = df5[df5['MAG_AUTO'] < 20]
df6f = df6[df6['MAG_AUTO'] < 20]

print('valid data points in first file:', len(df1f))
print('valid data points in second file:', len(df2f))
print('valid data points in third file:', len(df3f))
print('valid data points in fourth file:', len(df4f))
print('valid data points in fifth file:', len(df5f))
print('valid data points in sixth file:', len(df6f))

In [ ]:
match3(df1f, df2f, df3f) #finding common sources over the 3 inputs
print('There are', len(data1_123_matches), 'matches between the first 3 files')
plt.rcParams.update({'figure.max_open_warning': 0})

for i in range(len(data1_123_matches)): #looping over all matches
    
    script_dir = os.path.dirname(__file__) #this makes a directory to put all of the light curves into
    results_dir = os.path.join(script_dir, 'Lightcurves/')
    sample_file_name = "Lightcurve"
    if not os.path.isdir(results_dir):
        os.makedirs(results_dir)
    
    lc_1 = [] #dataframe for the points in the light curve
    lc_1 = pd.DataFrame(lc_1)
    lc_1 = lc_1.append(data1_123_matches.iloc[i]) #adding in the first data point
    lc_1 = lc_1.append(data2_123_matches.iloc[i]) # second 
    lc_1 = lc_1.append(data3_123_matches.iloc[i]) # third 
    mag_1 = lc_1['MAG_AUTO'] #this will be used for the light curve plot
    
    lc_2 = []
    lc_2 = pd.DataFrame(lc_2)
    match1(data1_123_matches.iloc[i,8:10], df4f)
    lc_2 = lc_2.append(catalogue_match)
    match1(data1_123_matches.iloc[i,8:10], df5f)
    lc_2 = lc_2.append(catalogue_match)
    match1(data1_123_matches.iloc[i,8:10], df6f)
    lc_2 = lc_2.append(catalogue_match)
    mag_2 = lc_2['MAG_AUTO']
    
    time = [8,9,10] # for the x axis of the light curve 

    fig, ax1 = plt.subplots(figsize = (6,6)) #light curve plotting paramters 
    plt.title('Light curve for subtraction data')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Magnitude in subtraction image', color = 'black')
    mag_1_plot = plt.plot(time, mag_1, color ='g', label = 'g')
    
    try: 
        mag_2_plot = plt.plot(time, mag_2, color ='grey', label = 'i')
    except ValueError:
        pass
    try: 
        time = [8,9]
        mag_2_plot = plt.plot(time, mag_2, color ='grey', label = 'i')
    except ValueError:
        pass
    try: 
        time = [8]
        mag_2_plot = plt.plot(time, mag_2, color ='grey', label = 'i')
    except ValueError:
        pass
    
    plt.locator_params(axis = 'y', nbins = 10)
    plt.locator_params(axis='x', nbins=10)
    plt.legend()
    
    plt.savefig(results_dir + sample_file_name + str(i) + '.png', format = 'png') #saving the light curve as an image
    
    if i % 25 == 0: #printing to see progress
        print('i = {}'.format(i))